In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import  r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
def load_data():
  # Loading of the dataset via pandas
  kc_data = pd.read_csv("../data/King_County_House_prices_dataset.csv")
  # We will drop this row
  kc_data.drop(15856, axis=0, inplace=True)
  return kc_data

In [3]:

# We replace "?" with Nan

class SqftColumnTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['sqft_basement'] = X['sqft_basement'].replace('?', np.NaN)
        # And we change the dtype of the column "sqft_basement" to float
        X['sqft_basement'] = X['sqft_basement'].astype(float)
        X.eval('sqft_basement = sqft_living - sqft_above', inplace=True)
        return X
    
class ViewColumnTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # We replace Nan values in "view" with the most frequent expression (0)
        X['view'].fillna(0, inplace=True)
        return X
    
class WaterFrontColumnTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # We replace Nan values in "waterfront" with the most frequent expression (0)
        X.waterfront.fillna(0, inplace=True)
        return X

class LastKnownChangeColumnTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # We will create an empty list in which we will store values
        last_known_change = []

        # For each row in our data frame, we look at what is in the column "yr_renovated".
        for idx, yr_re in X.yr_renovated.iteritems():
            # if "yr_renovated" is 0 or contains no value, we store the year of construction of the house in our empty listes ab
            if str(yr_re) == 'nan' or yr_re == 0.0:
                last_known_change.append(X.yr_built[idx])
            # if there is a value other than 0 in the column "yr_renovated", we transfer this value into our new list
            else:
                last_known_change.append(int(yr_re))
        # We create a new column and take over the values of our previously created list
        X['last_known_change'] = last_known_change
        return X

class DropExtraneousColumnsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # We delete the "yr_renovated" and "yr_built" columns
        X.drop("yr_renovated", axis=1, inplace=True)
        X.drop("yr_built", axis=1, inplace=True)
        return X

## Feature Engineering

In feature engineering, we try to generate additional features (variables) that have a better explanatory power or a higher predictive power with respect to our target variable.

In our case, we first want to adjust our target variable. Often people pay more attention to the price/performance ratio than to the price alone when making a purchase. In our example, we can achieve this by looking at the price per square foot of living space rather than the total price of the property.

In [4]:
# Feature engineering transformers

class SqFtPriceColumnTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # We create a new variable that gives us the price per square foot of living space
        X['sqft_price'] = (X.price/(X.sqft_living + X.sqft_lot)).round(2)
        return X

class CentreOfWealthColumnsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self
    
    def transform(self, X, y=None):
      # Absolute difference of latitude between centre and property
      X['delta_lat'] = np.absolute(47.62774- X['lat'])
      # Absolute difference of longitude between centre and property
      X['delta_long'] = np.absolute(-122.24194-X['long'])
      # Distance between centre and property
      X['center_distance']= ((X['delta_long']* np.cos(np.radians(47.6219)))**2 
                                   + X['delta_lat']**2)**(1/2)*2*np.pi*6378/360
      return X

class WaterDistanceColumnTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self
    
    # This function helps us to calculate the distance between the house overlooking the seafront and the other houses.
    def dist(self, long, lat, ref_long, ref_lat):
        '''dist computes the distance in km to a reference location. Input: long and lat of 
        the location of interest and ref_long and ref_lat as the long and lat of the reference location'''
        delta_long = long - ref_long
        delta_lat = lat - ref_lat
        delta_long_corr = delta_long * np.cos(np.radians(ref_lat))
        return ((delta_long_corr)**2 +(delta_lat)**2)**(1/2)*2*np.pi*6378/360

    def transform(self, X, y=None):
        water_distance = []
        water_list=  X.query('waterfront == 1')
        
        # For each row in our data frame we now calculate the distance to the seafront
        for idx, lat in X.lat.iteritems():
            ref_list = []
            for x,y in zip(list(water_list.long), list(water_list.lat)):
                ref_list.append(self.dist(X.long[idx], X.lat[idx],x,y).min())
            water_distance.append(min(ref_list))
        # wir erstellen eine neue Spalte und übernehmen die Werte unserer vorher erstellten Liste
        X['water_distance'] = water_distance
        return X
           

In [5]:
from sklearn.pipeline import Pipeline


class PreprocessingSeattleHousing():
    def __init__(self):
        # Data cleaning Pipeline
        self.data_cleaning_pipeline = Pipeline([
            ('sqft_basement', SqftColumnTransformer()),
            ('view', ViewColumnTransformer()),
            ('waterfront', WaterFrontColumnTransformer()),
            ('last_known_change', LastKnownChangeColumnTransformer()),
            ('drop_extraneous_columns', DropExtraneousColumnsTransformer())
        ])
        # Feature Engineering
        self.feature_enginneering = Pipeline([
            ('sqft_price', SqFtPriceColumnTransformer()),
            ('center_of_wealth', CentreOfWealthColumnsTransformer()),
            ('water_distance', WaterDistanceColumnTransformer())
        ])

        self.preprocessor_pipe = Pipeline([
            ('data_cleaning', self.data_cleaning_pipeline),
            ('feature_enginneering', self.feature_enginneering)
        ])

    def preprocess_fit_transform(self, df):
        return self.preprocessor_pipe.fit_transform(df)

    def preprocess_transform(self, df):
        return self.preprocessor_pipe.transform(df)

In [8]:
def train(dataset):
  drop_lst = ['price', 'sqft_price', 'date', 'delta_lat', 'delta_long',]
  # we would like to consider all variables except the ones mentioned above
  all_features = [x for x in dataset.columns if x not in drop_lst]
  # X contains all descriptive variables defined above
  X = dataset[all_features]
  # we define y (our dependent variable): we take the price
  print(X.describe())
  y = dataset['price']
  # We separate our data into train and test data. In the process, 30 % of the data is used for the subsequent testing of the prognostic quality.
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  # we can look at how much data is in each dataset
  print("X_train (features for the model to learn from): ", X_train.shape)
  print("y_train (labels for the model to learn from): ", y_train.shape)
  print("X_test (features to test the model's accuracy against): ", X_test.shape)
  print("y_test (labels to test the model's accuracy with): ", y_test.shape)
  # If we look at the first 5 lines of our training data, we see that the index is no longer sorted, it has been shuffled.
  X_train.head()

def main():
  df_dataset = load_data()

  pipeline = PreprocessingSeattleHousing()
  dataset = pipeline.preprocess_fit_transform(df_dataset)
  train(dataset)

main()

49        822039084
230      8096000060
246      2025069065
264      2123039032
300      3225069065
            ...    
19968    2025069140
20309     518500480
20751    8043700105
21185     518500460
21560    9253900271
Name: id, Length: 146, dtype: int64


In [32]:
def example_housing_data():
  csv_input = [
    ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'],
    ['7129300520', '10/13/2014', 221900.0, 3, 1.0, 1180, 5650, 1.0, None, 0.0, 3, 7, 1200, 0.0, 1955, 0.0, 98178, 47.5112, -122.257, 1340, 5650],
    ['7129300521', '12/13/2019', 250900.0, 3, 1.0, 1180, 5650, 1.0, None, 0.0, 3, 7, 1180,'?', 1955, 0.0, 98178, 47.5112, -122.257, 1340, 5650]
  ]
  return pd.DataFrame(csv_input[1:], columns=csv_input[0])
    
def expected_results():
  csv_results = [
    ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'],
    ['7129300520', '10/13/2014', 221900.0, 3, 1.0, 1180, 5650, 1.0, None, 0.0, 3, 7, 1180, 0.0, 1955, 0.0, 98178, 47.5112, -122.257, 1340, 5650],
    ['7129300521', '12/13/2019', 250900.0, 3, 1.0, 1180, 5650, 1.0, None, 0.0, 3, 7, 1180, 0.0, 1955, 0.0, 98178, 47.5112, -122.257, 1340, 5650]
  ]

result = example_housing_data().eval('sqft_basement = sqft_living - sqft_above')
print(result['sqft_basement'])

0   -20
1     0
Name: sqft_basement, dtype: int64


After these preparations, we now come to modelling. 
For this we will continue to use the scikit-learn library, in which many different algorithms are implemented.
The procedure is always the same:

- we import the algorithm from scikit-learn which we want to use.

- we determine the model, often there are additional hyperparameters we have to specify
- we determine which variables to pass to the model
- we train the model (we call the method `.fit(X_train, y_train)` on our model)
- we test the model with our test data and get the adjusted R^2 as metric (we call the method `.score(X_test, y_test)` on our trained model and clean up the score).


In [ ]:
# We determine the model, there must be 2 round brackets behind the model name!
model_lin_reg = LinearRegression()

Next, we determine which variables we pass to our model to determine the price of the houses.
The simplest model calculates the price using only one variable: for example, the "grade" of the house.

In [ ]:
# We determine which variables we pass to the model
variables = ['grade',]

In [ ]:
# Training of the model
model_lin_reg.fit(X_train[variables], y_train)

In [ ]:
# We look at how well our model performs on the test data
print('adj. R^2:', (1-(1-model_lin_reg.score(X_test[variables], y_test))*(X_test.shape[0]- 1)/(X_test.shape[0]-len(variables)-1)).round(2))

The adjusted R^2 indicates the percentage of variance of the target variable (price per square foot) explained by the model. Adjusted R² is a modified version of R² that has been adjusted with the number of explanatory variables. It penalises the addition of unnecessary variables and allows comparison of regression models with different numbers of explanatory variables.
The value 1 means 100 % of the variance of the target variable could be explained by the model. The value 0 means 0 % of the variance of the target variable could be explained by the model. 
This means for our case: The variable "grade" can explain 43 % of the variance in the price per square foot of the houses in our test set.
Perhaps more variables could explain more variance. 
We can look again at what variables we have:

In [ ]:
# Names of the variables in the data set
X_train.columns

We can use all these variables to predict the price per square foot.
Maybe the age of the house could also play a big influence.
Therefore, we will now try a new linear regression with these 2 variables.

In [ ]:
# We determine the model, there must be 2 round brackets behind the model name!
model_lin_reg = LinearRegression()

In [ ]:
# We determine which variables we pass to the model
variables = ['grade','last_known_change']

In [ ]:
# we train the model
model_lin_reg.fit(X_train[variables], y_train)

In [ ]:
# We look at how well our model performs on the test data
print('adj. R^2:', (1-(1-model_lin_reg.score(X_test[variables], y_test))*(X_test.shape[0]- 1)/(X_test.shape[0]-len(variables)-1)).round(2))

We see that with this additional variable, 48% of the variance in the price per square foot could be explained.

So far we have only looked at the linear relationships between the variables and the price. However, it is possible that the relationship is not linear, but rather quadratic. 
We can easily extend our model by squaring our variables. Thus, instead of:

$price = b*(grade) + c$

we can also use 

$price=a*(grade)^{2}+b*(grade)+c$

can be obtained.
This is a type of feature engineering. We will apply it to our complete data set and see if we can improve our model even further.

In [ ]:
# We want to create only polynomial variables of second order (^2)
poly = PolynomialFeatures(2)

In [ ]:
# create a copy of the Train and Test data
X_train_poly = X_train.copy()
X_test_poly = X_test.copy()

# drop the id column
X_train_poly = X_train_poly.drop(columns=['id'])
X_test_poly = X_test_poly.drop(columns=['id'])

In [ ]:
# We create new variables by calling poly
X_train_sq = poly.fit_transform(X_train_poly)

# We have to do the same for our test data, of course
X_test_sq = poly.transform(X_test_poly)

In [ ]:
# We determine the model, there must be 2 round brackets behind the model name!
model_lin_reg = LinearRegression()

In [ ]:
# We also train the model with squared variables
model_lin_reg.fit(X_train_sq, y_train)

In [ ]:
# We look at how well our model performs on the test data
print('adjusted R^2:', (1-(1-model_lin_reg.score(X_test_sq, y_test))*(X_test_sq.shape[0]- 1)/(X_test_sq.shape[0]-X_test_sq.shape[1]-1)).round(2))

With the additional squared variables, we were able to improve our result a bit more.


With the adjusted R^2 value we have a possibility to evaluate the quality of our model, but it may be worthwhile to have a look at the real errors of the model graphically. This may help to identify systematic errors.
For ease of interpretation, we choose the percentage price difference between our forecast and the true values.

We see a few outliers here. We can take a closer look at the highest one. 

In [ ]:
# Error analysis
# In order to better analyse the errors of our model, we create a new dataframe with the
# columns "price" (the real price), as well as the latitudes and longitudes
y_predictions = model_lin_reg.predict(X_test_sq)
df_error = pd.DataFrame(y_test)
df_error['latitude'] = X_test['lat']
df_error['longitude'] = X_test['long']
df_error['id'] = X_test['id']
df_error.head(2)

In [ ]:
# To add the predicted price as a column as well, we must first reset the index
df_error.reset_index(inplace=True, drop=True)
df_error.head(2)

In [ ]:
# Now we can also add the predicted price as a column and calculate the difference
df_error['price_prediction'] = y_predictions.round(2)
df_error['price_difference'] = (df_error['price_prediction'] - df_error['price']).round(2)
df_error['price_difference_procent'] = ((df_error['price_difference']/df_error['price'])*100).round(2)
df_error.head(2)

In [ ]:
fig = px.scatter_mapbox(df_error,
                        lat="latitude",
                        lon="longitude",
                        hover_data=["price", "price_prediction", 'id'],
                        color='price_difference_procent',
                        color_continuous_scale=['green', 'yellow', 'red'],
                        zoom=7.7,
                        height=400)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
df_error[df_error['price_difference_procent']==df_error['price_difference_procent'].max()]

In [ ]:
X_test[X_test['id']==9272202260]

We want to take a closer look at these houses. King County also provides very good information on this. On [this page](https://localscape.property/#kingcountyassessor/My-Property) you can search for houses by their ID and get both the neighbourhood on a map and a picture of the house.
In the field at the top left, change the selection "Address" to "Parcel ID" and add the "id" of our outlier.

Under "Basic Property Characteristics" and on the map under "KC Aerial Images" we see that there is no longer a house on this property. Therefore, our data is misleading and our model estimates a much higher price.


## Regularisation and hyperparameter tuning of linear regression

In addition to our variables, we have also passed the squared variables to our last linear model. So we have passed a lot of variables to our model. Some may have no effect on the price at all. However, models try to extract some information from all variables. This leads to random variance in the data also being recognised as a pattern. This phenomenon is called "overfitting" the model to the data.
For each algorithm there are ways to prevent this overfitting.

In the case of linear regression, we force the model not to use variables for forecasting. We "regularise" the model. But instead of us telling the model which variables not to use, we let the model learn which variables offer the least added value and remove those variables (in the case of linear regression, variables are no longer considered if the learned coefficient (b) is zero). How much we regularise is up to us.
[Here](https://scikit-learn.org/stable/modules/linear_model.html#elastic-net) you can find information on the ElasticNet model used.
To find out how much we should regularize our model, we can test different values for the regularzation parameters and see which will give us the best model. For this purpose we can use [GridSearch](https://scikit-learn.org/stable/modules/grid_search.html#grid-search) with [Cross-Validation](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation).



In [ ]:
# We specify which hyperparameters we want to change:
# alpha: specifies how much we regularise:
param_grid = {'alpha':[0.1,.5,1,5,10,],
             'l1_ratio': [1,0.5,0]}

# We determine the model, there must be 2 round brackets behind the model name!
elastic = ElasticNet(max_iter=50000, tol=0.2)

# Passing the model to a so-called parameter grid with 5-fold cross-validation
elastic_cv= GridSearchCV(elastic,param_grid,cv=5, verbose=True,n_jobs=-1)

# We train the model and optimise it via GridSearch
elastic_cv.fit(X_train_sq,y_train)

In [ ]:
# Output of the best parameters found by the GridSearch.
elastic_cv.best_params_

In [ ]:
# We train the model with the optimal hyperparameters
elastic = ElasticNet(max_iter=50000, tol=0.2,**elastic_cv.best_params_)

elastic.fit(X_train_sq, y_train)

In [ ]:
# We look at how well our model performs on the test data
adj_r2 = (1-(1-elastic.score(X_test_sq, y_test))*(X_test_sq.shape[0]- 1)/(X_test_sq.shape[0]-(X_test_sq.shape[1]-sum(elastic.coef_== 0))-1)).round(2)
print('adjusted R^2:',adj_r2 )

We were able to further improve our model through hyperparameter tuning and regularisation. 
As mentioned above, regularisation eliminates variables from the forecast. This is done by giving the coefficients of the linear regression a value of zero. With the following code we can look at the learned coefficients (here only a section of the first 5 coefficients) and see that for the first variable a coefficient of zero was calculated. This variable was therefore removed by our regularisation.

In [ ]:
# Output of the first 5 learned coefficients of linear regression
elastic.coef_[0:5]

In [ ]:
# Error analysis
# In order to better analyse the errors of our model, we create a new dataframe with the
# columns "price" (the real price), as well as the latitudes and longitudes
y_predictions = elastic.predict(X_test_sq)
df_error = pd.DataFrame(y_test)
df_error['latitude'] = X_test['lat']
df_error['longitude'] = X_test['long']
df_error['id'] = X_test['id']
df_error.head(2)

In [ ]:
# To add the predicted price as a column as well, we must first reset the index
df_error.reset_index(inplace=True, drop=True)
df_error.head(2)

In [ ]:
# Now we can also add the predicted price as a column and calculate the difference 
df_error['price_prediction'] = y_predictions.round(2)
df_error['price_difference'] = (df_error['price_prediction'] - df_error['price']).round(2)
df_error['price_difference_procent'] = ((df_error['price_difference']/df_error['price'])*100).round(2)
df_error.head(2)

In [ ]:
fig = px.scatter_mapbox(df_error,
                        lat="latitude",
                        lon="longitude",
                        hover_data=["price", "price_prediction", 'id'],
                        color='price_difference_procent',
                        color_continuous_scale=['green', 'yellow', 'red'],
                        zoom=7.7,
                        height=400)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
df_error[df_error['price_difference_procent']==df_error['price_difference_procent'].max()]

In [ ]:
X_test[X_test['id']==5111400086]

Let's take a closer look at this house. King County also provides very good information about it. On [this page](https://localscape.property/#kingcountyassessor/My-Property) you can search for houses by their ID and get both the neighbourhood on a map and a picture of the house.
In the box at the top left, they change the "Address" selection to "Parcel ID" and add the "id" of our outlier there.

This house sold in 2018 for a price of  295,000 USD. But King County appraisers also assessed a higher price in 2014: under "Historical Value" they see that in 2014 the "Total Assessed Value" was 212,000 USD. So this house sold for about half the appraised price!



## Save the model

We have now trained a model that can predict the price of a house in King County. We can now save this model using [skops](https://skops.readthedocs.io/en/stable/modules/classes.html#module-skops.io).

In [ ]:
import skops.io as sio

with open('model/model.bin', 'wb') as f_out:
    sio.dump(elastic, f_out)

### Conclusion
Despite the outliers, we succeeded in creating a model that predicts prices with an accuracy of 76%. We found that the creation of new variables, but also the squaring of these variables and the regularisation of the model play an important role in the quality of the prediction. 